In [ ]:
%pip install torch tqdm scikit-learn seaborn pandas

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

EMBEDDING_DIM = 128
NUM_FILTERS = 64
FILTER_SIZE = 8
HIDDEN_UNITS = 16
BATCH_SIZE = 8
EPOCHS = 10
LEARNING_RATE = 1e-3
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

EMBEDDING_DIM = 128
NUM_FILTERS = 64
FILTER_SIZE = 8
HIDDEN_UNITS = 16
BATCH_SIZE = 8
EPOCHS = 10
LEARNING_RATE = 1e-3
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

class OpcodeDataset(Dataset):
    def __init__(self, opcodes, labels, vocab, max_length=666277):
        self.opcodes = opcodes
        self.labels = labels
        self.vocab = vocab
        self.max_length = max_length

    def __len__(self):
        return len(self.opcodes)

    def __getitem__(self, idx):
        op_seq = self.opcodes[idx]
        label = self.labels[idx]
        indices = [self.vocab.get(op, 0) for op in op_seq.split()[:self.max_length]]
        return torch.tensor(indices, dtype=torch.long), torch.tensor(label, dtype=torch.long)

def collate_fn(batch):
    sequences, labels = zip(*batch)
    sequences = pad_sequence([seq.clone().detach() for seq in sequences], batch_first=True, padding_value=0)
    labels = torch.tensor(labels, dtype=torch.long)
    return sequences, labels

class OpcodeCNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_filters, filter_size, hidden_units, num_classes):
        super(OpcodeCNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.conv = nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters, kernel_size=filter_size, padding=filter_size // 2)
        self.relu = nn.ReLU()
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.fc1 = nn.Linear(num_filters, hidden_units)
        self.fc2 = nn.Linear(hidden_units, num_classes)

    def forward(self, x):
        x = self.embedding(x).permute(0, 2, 1)
        x = self.relu(self.conv(x))
        x = self.pool(x).squeeze(2)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def train_model(model, train_loader, val_loader, num_epochs, optimizer, criterion):
    model.to(DEVICE)
    best_val_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        val_loss, val_acc = evaluate_model(model, val_loader, criterion)
        print(f"Epoch {epoch+1}: Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_model.pth")

def evaluate_model(model, data_loader, criterion):
    model.eval()
    total_loss = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    return total_loss / len(data_loader), acc

def generate_reports(model, data_loader, class_names):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print("Accuracy:", accuracy_score(all_labels, all_preds))

    print("Classification Report:\n")
    print(classification_report(all_labels, all_preds, labels=range(len(class_names)), target_names=class_names, zero_division=0))

    conf_matrix = confusion_matrix(all_labels, all_preds, labels=range(len(class_names)), normalize="true")

    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=False, fmt=".2f", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title("Normalized Confusion Matrix")
    plt.show()

def main():
    data = pd.read_csv("output.csv")
    opcodes = data["Opcodes"].tolist()
    labels = data["APT"].tolist()

    label_mapping = {label: idx for idx, label in enumerate(sorted(set(labels)))}
    labels = [label_mapping[label] for label in labels]
    class_names = [str(key) for key in label_mapping.keys()]

    vocab = {word: idx + 1 for idx, word in enumerate(set(" ".join(opcodes).split()))}
    vocab_size = len(vocab) + 1

    train_opcodes, val_opcodes, train_labels, val_labels = train_test_split(opcodes, labels, test_size=0.2, random_state=6530)

    print("Number of training samples:", len(train_opcodes))
    print("Number of validation samples:", len(val_opcodes))

    train_dataset = OpcodeDataset(train_opcodes, train_labels, vocab, max_length=666277)
    val_dataset = OpcodeDataset(val_opcodes, val_labels, vocab, max_length=666277)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

    print("Sample data shape:")
    for inputs, labels in train_loader:
        print("Inputs shape:", inputs.shape)
        print("Labels shape:", labels.shape)
        break

    num_classes = len(label_mapping)
    model = OpcodeCNN(vocab_size, EMBEDDING_DIM, NUM_FILTERS, FILTER_SIZE, HIDDEN_UNITS, num_classes)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.CrossEntropyLoss()

    train_model(model, train_loader, val_loader, EPOCHS, optimizer, criterion)

    model.load_state_dict(torch.load("best_model.pth"))
    val_loss, val_acc = evaluate_model(model, val_loader, criterion)
    print(f"\nBest Validation Loss: {val_loss:.4f}, Best Validation Accuracy: {val_acc:.4f}")

    generate_reports(model, val_loader, class_names)

if __name__ == "__main__":
    main()
